# 5. 📉 KNN:
Exported from Filament on Thu, 17 Mar 2022 19:30:23 GMT

---

I also experimented with KNN Clustering, just to see if I could improve on the 2 false negatives I got from my final Decision Tree. First lets run the model on all the features and see what we get, before refining down

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

##
from sklearn import datasets
import sklearn.metrics as sm
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
df = pd.read_csv('cleaned_mushrooms.csv') # reading in the cleaned mushroom data

## ✅ Label Encoding:

In [ ]:
# cols with only two categories can be label encoded:

for col in df.columns:
    if len(df[col].value_counts()) == 2:
        df[col] = df[col].astype('category') ## change these cols to category data types
        df[col] = df[col].cat.codes ## use cat.codes function to label encode

df.dtypes

In [ ]:
for col in df.columns:
    print(col, " : ", df[col].unique()) # just displays all the cols and their unique values

## ✅ OHE:

In [ ]:
# here we one hot encode all the cols that haven't already been label encoded

one_hot = list(df.columns)
one_hot.remove('class')
one_hot.remove('bruises')
one_hot.remove('gill_attachment')
one_hot.remove('gill_spacing')
one_hot.remove('gill_size')
one_hot.remove('stalk_shape')
df = pd.get_dummies(df, columns = one_hot, prefix = one_hot)

In [ ]:
df.shape

In [ ]:
# thought it would be useful to save here in case I refine my KNN model features

df.to_csv('knn_mushrooms.csv', index = False)

## ✅ Train/Test Split:

In [ ]:
# here we train/test split with tes size being 20%, and set a random state of 124
# we also stratify to ensure the best representation is being provided
X = df.drop(columns=['class'])
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=124, stratify=y)

In [ ]:
# we scale the data to make it easier for our model to interpret

scaler = MinMaxScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# here we're displaying the error rate in a random range, in order to idnetify what the best n-neighbors parameter is
errors = []

for k in range(1, 100):
    knn = KNeighborsClassifier(n_neighbors= k)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    errors.append(np.mean(pred_i != y_test))
    
plt.figure(figsize=(12, 6))
plt.plot(range(1, 100)
         , errors
         , color='black'
         , linestyle='dashed'
         , marker='o'
         , markerfacecolor='grey'
         , markersize=10)
plt.title('Error Rate K Value')
plt.xlabel('K Value')
plt.ylabel('Mean Error')
plt.show()

In [ ]:
# we can see from the plot above that 10 is the lowest mean error rate aove zero- great
# lets run the model

classifier = KNeighborsClassifier(n_neighbors= 10)
classifier.fit(X_train, y_train)

In [ ]:
# creating a prediction and producing a confusion matrix and classification report
# we can see the results are very similar to the original decision tree model, just 1 false negative

y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Lets refine down the features now using the same list we generated from the feature importance and try it with KNN modelling- see "6. KNN- Final"